# Look at our Evaluations

In [1]:
%load_ext autoreload

# Automatically reload all modules (except those excluded by %aimport)
%autoreload 2

In [2]:
import sys
import utils

In [39]:
data = utils.read_dataset('FB15K-237-N')

2024-03-08 16:02:34,865 utils INFO: Reading dataset FB15K-237-N...
2024-03-08 16:02:34,865 utils INFO: Reading dataset FB15K-237-N...
2024-03-08 16:02:34,865 utils INFO: Reading dataset FB15K-237-N...
2024-03-08 16:02:34,865 utils INFO: Reading dataset FB15K-237-N...
2024-03-08 16:02:34,865 utils INFO: Reading dataset FB15K-237-N...
2024-03-08 16:02:34,865 utils INFO: Reading dataset FB15K-237-N...
2024-03-08 16:02:34,865 utils INFO: Reading dataset FB15K-237-N...
Saved to f'pos_tmp.jsonl
Saved to f'neg_tmp.jsonl


In [3]:
save_path = '/home/jackboylan/knowledge-base-builder/data/results/CoDeX-S/WikipediaKGValidator/50_WikipediaKGValidator_CoDeX-S_seed23_gpt-3.5-turbo-0125.jsonl'


In [26]:
# import duckduckgo_verbose_search
# import json

# query = "What Roy Marlin Voris nationality Tahiti?"

# web_searcher = duckduckgo_verbose_search.DuckDuckGoVerboseSearch(max_search_results=5)
# web_content = web_searcher(query)
# print(json.dumps(web_content, indent=2))

In [27]:
results = utils.read_jsonl(save_path)
len(results)

2

In [28]:
tp = 0
fp = 0
tn = 0
fn = 0 
pos_results = results[1]
neg_results = results[0]
for val in pos_results['validated_triples']:
    if val['triple_is_valid']:    # property is correctly marked as valid
        tp += 1
    else:                           # property is incorrectly marked as invalid
        fn += 1
for val in neg_results['validated_triples']:
    if val['triple_is_valid']:    # property is incorrectly marked as valid
        fp += 1
    else:                           # property is correctly marked as invalid
        tn += 1

metrics = utils.calc_metrics(tp, fp, tn, fn)

# Precision: 0.75
# Recall: 0.8571428571428571
# F1 Score: 0.7999999999999999
# Accuracy: 0.7857142857142857
# ----------

Precision: 0.88
Recall: 0.88
F1 Score: 0.88
Accuracy: 0.88
----------


In [29]:
# what did the evaluator get wrong?
import json
false_neg = 0
false_pos = 0
unsure = 0
print('\n----------------False Negatives------------------\n')
for val in pos_results['validated_triples']:
    if val['triple_is_valid'] is False:
        print(json.dumps(val, indent=2))
        false_neg += 1
print(f"Total False Negatives: {false_neg}")


print('\n----------------False Positives------------------\n')
for val in neg_results['validated_triples']:
    if val['triple_is_valid'] is True:
        print(json.dumps(val, indent=2))
        false_pos += 1
print(f"Total False Positives: {false_pos}")


print('\n----------------Unsure (Not enough info to say)------------------\n')
for val in neg_results['validated_triples']:
    if val['triple_is_valid'] not in [True, False]:
        print(json.dumps(val, indent=2))
        unsure += 1
print(f"Total Unsure: {unsure}")




----------------False Negatives------------------

{
  "predicted_subject_name": "Syria",
  "predicted_relation": [
    "diplomatic relation"
  ],
  "predicted_object_name": "Japan",
  "triple_is_valid": false,
  "reason": "The predicted triple is not valid because there is no direct mention or indication of diplomatic relations between Syria and Japan in the provided context. The context primarily focuses on Syria's diplomatic relations with countries in its region, the Arab League, and the Western world, but Japan is not mentioned in any of these contexts.",
  "sources": {
    "relevant_text": [
      "Since the ongoing civil war of 2011, and associated killings and human rights abuses, Syria has been increasingly isolated from the countries in the region, and the wider international community. Diplomatic relations have been severed with several countries including: Britain, Canada, France, Italy, Germany, Tunisia, Egypt, Libya, the United States, Belgium, Spain, and the Arab states

In [30]:
print(f"Total Unsure: {unsure}")
print(f"Total False Negatives: {false_neg}")
print(f"Total False Positives: {false_pos}")

Total Unsure: 3
Total False Negatives: 3
Total False Positives: 0


In [31]:
for val in neg_results['validated_triples']:
    if val['triple_is_valid'] is False:
        print(json.dumps(val, indent=2))

{
  "predicted_subject_name": "Xavier Cugat",
  "predicted_relation": [
    "country of citizenship"
  ],
  "predicted_object_name": "Canada",
  "triple_is_valid": false,
  "reason": "Xavier Cugat's known countries of citizenship are Spain, Cuba, and the United States of America, as per the provided Wikidata reference. There is no mention or indication of Canadian citizenship in the context or the reference. Therefore, the predicted triple is not valid in this case.",
  "sources": {
    "relevant_text": [
      "time after his stay in Marseille, Tzara joined the French Resistance, rallying with the Maquis. A contributor to magazines published by the Resistance, Tzara also took charge of the cultural broadcast for the Free French Forces clandestine radio station. He lived in Aix-en-Provence, then in Souillac, and ultimately in Toulouse. His son Cristophe was at the time a Resistant in northern France,",
      "for the Free French Forces clandestine radio station. He lived in Aix-en-Prov

In [35]:
umls_relation_dict = json.load(open('../data/UMLS/umls_relation_dict.json', 'r'))